In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

import matplotlib as mpl
import matplotlib.colors as mcolors  
import matplotlib.pyplot as plt      

from matplotlib import rc 

In [3]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b k_B}`
        
    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        
    Returns:
        p_c: Critical pressure in Pa.\n
        v_c: Critical volume in m^3/mol.\n
        T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/k_B/N_A
    
    return p_c, v_c, T_c

In [4]:
def isotherms(a, b, T_range, v_range):
    
    """
    
        This function calculates the p(v, T) plane
        from given a and b parameters of 
        the Van der Waals equation of state for real gases.

    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        T_range: Tuple containing maximum and minimum values of
        temperature to be plotted. Temperatures must be expressed
        in terms of the critical temperature.\n
        v_range: Tuple containing maximum and minimum values of
        volumen to be plotted. Volumens must be expressed
        in exponents of the critical volumen (v in 10^v_range).\n
        
    Returns:
        isotherms: a dictionary containing the values of v and the
        pressures of the isotherms in the following form:\n
            isotherms['v'] = np.array containing the values of v
            in a logarithmic scale.\n
            isotherms['value of the isotherm'] = np.array containing
            the values of p.\
    """
    

    
    if b != 0.0:
        p_c, v_c, T_c = calculate_critic(a, b)
        
        isotherms = {}
        v_R = np.logspace(min(v_range), max(v_range), num = 500, base = 10.0)
        isotherms['v'] = v_R
        
        for T in T_range:
            p_R = []
            for v in v_R:
                val = (8.0/3.0*T/(v - 1.0/3.0) - 3.0/v**2)
                p_R = np.append(p_R, val)

            isotherms['%.2f' % T] = p_R

    return isotherms

In [5]:
def update_isotherm(change):
    
    """    
        This function makes the isotherm expressed by a
        float ipython widget visible in the 'fig' plot and
        hides the rest of the lines (the ones in the list 
        'saved' are always displayed).\n

    Args:
        change: variable returned by my_float_widget.observe(update
        _isotherm, 'value').\n

    """
    
    temp = np.arange(T_r.min, T_r.max+0.05, 0.05)
    t_range = np.around(temp, 2)
    
    old, = np.where(t_range == float(change.old))[0]
    i, = np.where(t_range == float(change.new))[0]
    
    if old not in saved:
        fig.marks[old].visible = False
        fig.marks[old].display_legend = False
        
    fig.marks[i].visible = True
    fig.marks[i].display_legend = True
    
    update_tracer(change)

In [6]:
def save_isotherm(T):
    
    """    
        This function adds the isotherm expressed by a
        float ipython widget ('T_r') visible to the 'saved' list
        (the ones in the list 'saved' are always displayed
        in the 'fig' plot).\n

    Args:
        a: variable returned by save_button.on_click(save_isotherm)
        .\n

    """
    i = get_isotherm_index(T)
    saved.append(i)    
    fig.marks[i].visible = True  #trivial when called by 'save isotherm button' but allows to 'show all checkbox' work

In [7]:
def show_isotherm(T):
    i = get_isotherm_index(T)
    fig.marks[i].visible = True

In [8]:
def get_isotherm_index(T):
    t_range = get_t_range(t_min_input, t_max_input) 
    i, = np.where(t_range == float(T))[0]
    return i

In [9]:
def save_button_clicked(a):
    save_isotherm(T_r.value)

In [10]:
def undo(change):
    
    """    
        This function pops the isotherm expressed by a
        float ipython widget ('T_r') visible from the 'saved' list
        (the ones in the list 'saved' are always displayed
        in the 'fig' plot).\n

    Args:
        change: variable returned by undo_button.on_click(undo)
        .\n

    """
        
    if len(saved) > 0:
        popped = saved.pop(-1)
        i, = np.where(t_range == float(T_r.value))[0]
        if i != popped:
            fig.marks[popped].visible = False

In [11]:
def save_png(change):
    
    """
    
        This function calls the matplotlib_figure() function
        and if the is no errors creates an HTML button to download
        the file.\n

    Args:
        change: variable returned by export_png_button.on_click(save_png).\n
        
    """
    
    try:
        png_created_text.value ="<b>Creating PNG file...</b>"
        filename = matplotlib_figure(pdf = False, png = True, fig = fig)
        png_created_text.value = "<span style='color:green'><b>PNG file '" + filename + "' successfully created, download it here:</b></span><form action=" + filename + " target='_blank'><button type=''submit''>Download PNG</button></form>"

    except:
        png_created_text.value = "<span style='color:red'><b>An error has occurred</b></span>"

In [12]:
def update_data(change):
    data = isotherms(a.value,b.value, t_range, v_limits)

In [13]:
def update_critics(change):
    
    if b.value == 0.0:
        return None
    
    p_c, v_c, T_c = calculate_critic(a.value, b.value * 0.001)
    
    p_c_text.value = '%.2f' % (p_c / 1000000.0) #Show in MPa
    v_c_text.value = '%.2f' % (v_c * 1000000.0)
    T_c_text.value = '%.2f' % (T_c)

In [14]:
def get_t_range(t_min_input, t_max_input, num_input):
    
    t_min = t_min_input.value
    t_max = t_max_input.value
    num = num_input.value
    
    if t_min > t_max or t_min == t_max:
        return None
    
    return np.linspace(start=t_min, stop=t_max, num=num)

In [15]:
def update_t_range(change):
    
    t_range = get_t_range(t_min_input, t_max_input)
    
    T_r.min = min(t_range)
    T_r.max = max(t_range)

    data = isotherms(a.value, b.value * 0.001, t_range, v_limits)
    
    #isotherm_lines = []
    marks = []
    size  = len(data.keys())
    step = np.floor_divide([255], [size])[0]

    c1 = 255
    c2 = 0
    c3 = 0
    
    vis = str(T_r.value)
    
    if  vis not in data.keys():
        
        vis = '1.00'

        if vis not in data.keys():
            vis = min(data.keys())

    T_r.value = float(vis)
    
    for line in fig.marks:
        line.visible = False
    
    saved.clear()
    
    for t in data.keys():
        if t != 'v':

            color = '#%02x%02x%02x' % (c1, c2, c3)

            marks.append(bqm.Lines(
            #isotherm_lines.append(bqm.Lines(
                x = data.get('v'), 
                y = data.get(t), 
                scales = {'x': scale_x, 'y': scale_y}, 
                #opacities = [1.0],
                visible = t == vis,
                display_legend = t == vis,
                colors = [color],
                labels = ['T = ' + t],
                ))
            c2 = c2 + step
            

    marks.append(tracer)
    marks.append(labels_points)
    marks.append(isopressure)
       
    maxwell_construction_checkbox.value = False
    show_tracer_checkbox.value = False
    
    fig.marks = marks

In [16]:
def show_all(change):
    if show_all_checkbox.value:
        
        T_r.disabled = True
        
        for t in get_t_range(t_min_input, t_max_input):
            show_isotherm(t)
            
    else:
        
        T_r.disabled = False
        
        for j in range(len(fig.marks)):
            if j not in saved and fig.marks[j].name not in ('Scatter'):
                fig.marks[j].visible = False
        
        tracer.visible = True
        isopressure.visible = True
        i = get_isotherm_index(T_r.value)
        fig.marks[i].visible = True

In [17]:
def find_nearest_index(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [18]:
def show_tracer(change):
    if show_tracer_checkbox.value:
        tracer.visible = True
        tracer_v.disabled = False
        update_tracer(change)
        
        label_input.disabled = False
        add_label_button.disabled = False
    else:
        tracer.visible = False
        tracer_v.disabled = True
        
        label_input.disabled = True
        add_label_button.disabled = True

In [19]:
def update_tracer(change):
    i_t = get_isotherm_index(T_r.value)
    i_v = find_nearest_index(fig.marks[i_t].x, tracer_v.value)
    tracer.x = np.array([fig.marks[i_t].x[i_v]])
    tracer.y = np.array([fig.marks[i_t].y[i_v]])

In [20]:
def add_label(x, y, label='froga'):
    
    x_labels = [elem for elem in labels_points.x]
    y_labels = [elem for elem in labels_points.y]
    
    x_labels.append(tracer.x[0])
    y_labels.append(tracer.y[0])
    
    labels_points.x, labels_points.y = x_labels, y_labels
    labels_points.names = np.append(labels_points.names, label)

In [21]:
def get_new_label(labels):
    alpha = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    alpha2 = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    count = 0
    i = 0
    while i < len(alpha):
        
        elem = alpha[i]
        if not np.isin(elem,labels):
            return elem
        
        if i == (len(alpha) - 1):
            alpha = [alpha2[count] + elem for elem in alpha2]
            i = 0
            count = count + 1
            
        i = i + 1

In [22]:
def add_label_button_clicked(a):
    label = label_input.value
    if label == '':
        label = get_new_label(labels_points.names)
    add_label(tracer.x[0], tracer.y[0], label)
    label_input.value = ''

In [23]:
def show_maxwell(change):
    
    if maxwell_construction_checkbox.value:
        maxwell_p.disabled = False
        shade_areas_checkbox.disabled = False
        
        isopressure.visible = True
        update_isopressure(change)
        
    else:
        maxwell_p.disabled = True
        shade_areas_checkbox.disabled = True
        isopressure.visible = False

In [24]:
def update_isopressure(change):

    i = get_isotherm_index(T_r.value)
    x_values = fig.marks[i].x
    y_values = fig.marks[i].y
    
    p = maxwell_p.value
    T = T_r.value
    
    roots = np.roots([1.0, - 1.0/3.0*(1.0 + 8.0*T/p), 3.0/p, -1.0/p])
    roots_in_range = []
    
    for root in roots:
        if np.isreal(root):
            root = np.real(root)
            if root < max(x_values) and root > min(x_values):
                roots_in_range.append(root)
    #print(roots_in_range) 
    roots_in_range.sort()
    
    if len(roots_in_range) == 3:
        isopressure.fill = 'between'
        shade_areas(change)
        
        i_min = find_nearest_index(x_values, roots_in_range[0])
        i_max = find_nearest_index(x_values, roots_in_range[2])
        x_real = x_values.tolist()[i_min:i_max]
        x = [x_real, x_real]
        y = [[maxwell_p.value for elem in x_real], fig.marks[i].y[i_min:i_max]]

        left_integral = defined_integral(T, roots_in_range[0], roots_in_range[1], p)
        right_integral = defined_integral(T, roots_in_range[1], roots_in_range[2], p)
        
        integral_value_left_text.value = '%.2f' % left_integral
        integral_value_right_text.value = '%.2f' % right_integral
        
        integral_value_text.value = '%.1f' % (left_integral + right_integral)
        
    else:
        isopressure.fill = 'none'
        x = [x_values, x_values]
        y = [[maxwell_p.value for elem in x_values], fig.marks[i].y]
        
        integral_value_left_text.value = '-'
        integral_value_right_text.value = '-'
        integral_value_text.value = '-'

    isopressure.x, isopressure.y = x, y

In [25]:
def shade_areas(change):
    
    if shade_areas_checkbox.value:
        
        isopressure.fill_opacities = [0.35]
        
    else:
        
        isopressure.fill_opacities = [0.0]

In [26]:
def p_undefined_integral(T, v_0, p_0):
    
    return 8.0/3.0 * T *np.log(v_0 - 1.0/3.0) + 3.0/v_0 - p_0*v_0

In [27]:
def defined_integral(T, v_0, v_1, p_0):
    
    return p_undefined_integral(T, v_1, p_0) - p_undefined_integral(T, v_0, p_0)

In [28]:
def find_real_isotherm(a):
    
    maxwell_p.continuous_update = False
    
    p_min = maxwell_p.min
    p_max = maxwell_p.max
    step = maxwell_p.step
    
    p_range = np.arange(p_max, p_min, -step)
       
    for p in p_range:
        maxwell_p.value = p
        if integral_value_text.value == '0.0':
            break
            

In [29]:
def prepare_export(a):
    global fig_str
    fig_str = repr(fig)
    %store fig_str
    export_button.value = "<form action=" + "../../../apps/modules/export_module.ipynb" + " target='_blank'><button type=''submit''>Open in export module</button></form>"


In [36]:
"""

.. module:: p_v_2D.ipynb
    :sypnopsis: This module creates an interface to interact with the
    Van der Waals isotherms in the p(v, T) plane.\n

.. moduleauthor:: Jon Gabirondo López (jgabirondo001@ikasle.ehu.eus)

"""

#%store -d fig_str
fig_str = ''

saved = []

top_block = widgets.VBox([])

t_min_input = widgets.BoundedFloatText(
    value=0.5,
    min=0.5,
    max=3.0,
    step=0.005,
    description=r'\( T_{min}: \)',
    disabled=False
)

t_max_input = widgets.BoundedFloatText(
    value=2.0,
    min=0.5,
    max=3.0,
    step=0.005,
    description=r'\( T_{max}: \)',
    disabled=False
)

#t_min_input.observe(update_t_range, 'value')
#t_max_input.observe(update_t_range, 'value')

num_input = widgets.IntSlider(
        value=5,
        min=1,
        max=20,
        description=r'\( \# \)',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        #readout_format='.2f',
    )

generate_button = widgets.Button(
        description='Generate',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Generate isotherms in the selected T range.',
    )

top_block.children = (widgets.HBox([
                        t_min_input,
                        t_max_input,
                        num_input,
                        generate_button]),)

#t_limits = (0.9, 1.0)
v_limits = (-0.4, 2.0) #v in 10^v_limits range
t_range = get_t_range(t_min_input, t_max_input, num_input)

T_c_index, = np.where(t_range == 1.0)[0]
data = isotherms(a.value,b.value * 0.001, t_range, v_limits)

scale_x = bqs.LogScale(min = min(data.get('v')), max = max(data.get('v')))
scale_y = bqs.LinearScale(min = min(data.get('1.00'))-0.3, max = 3.00)

fig = bq.Figure(title='Van der Waals isotherms',
                marks=[],
                axes=[],
                animation_duration=500,
                layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=80, bottom=80, left=75, right=30)
    )

axis_x = bqa.Axis(scale=scale_x,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                ticks=[0,10,100],
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v [log]',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(scale=scale_y,
                tick_format='0.2f',
                tick_style={'font-size': '15px'},
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig.axes = [axis_x, axis_y]

#isotherm_lines = []
marks = []
size = len(data.keys()) - 1
step = int(np.around([255/size], 0)[0])

c1 = 255
c2 = 0
c3 = 0

for t in data.keys():
    if t != 'v':

        color = '#%02x%02x%02x' % (c1, c2, c3)

        marks.append(bqm.Lines(
            x = data.get('v'), 
            y = data.get(t), 
            scales = {'x': scale_x, 'y': scale_y}, 
            #opacities = [1.0],
            visible = t == '1.00',
            colors = [color],
            labels = ['T = ' + t],
            ))
        c2 = c2 + step

tracer = bqm.Scatter(
    name = 'tracer',
    x = [1.0],
    y = [1.0],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = False,
    colors = ['black'],
    names = [],
)        

marks.append(tracer)

labels_points = bqm.Scatter(
    name = 'labels',
    x = [],
    y = [],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
    names = [],
) 

marks.append(labels_points)

isopressure = bqm.Lines(
            x = [], 
            y = [], 
            scales = {'x': scale_x, 'y': scale_y}, 
            opacities = [1.0, 0.0],
            visible = False,
            colors = ['green'],
            fill_opacities = [0.35],
            fill = 'none'
            )

marks.append(isopressure)
#labels = bqm.Label(
#    x=[],
#    y=[],
#    scales={'x': scale_x, 'y': scale_y},
#    text=[],
#    default_size=14,
#    font_weight='bolder',
#    colors=['black'],
#    update_on_move=True,
#    enable_move = True)
        
#marks.append(labels)    
    
fig.marks = marks #isotherm_lines
fig.marks[T_c_index].visible = True
fig.marks[T_c_index].display_legend = True

p_c_text = widgets.HTML(
    value="",
)

v_c_text = widgets.HTML(
    value="",
)
T_c_text = widgets.HTML(
    value="",
)

update_critics(None)

T_r = widgets.FloatSlider(
        value=1.0,
        min=min(t_range),
        max=max(t_range),
        step=0.05,
        description=r'\( \frac{T}{T_c} \)',
        disabled=False,
        continuous_update=True,
        orientation='vertical',
        readout=True,
        readout_format='.2f',
        layout = widgets.Layout(height = '80%', margin = '45px 0 0 0')
    )

T_r.observe(update_isotherm, 'value')

show_all_checkbox = widgets.Checkbox(
    value=False,
    description='Show all',
    disabled=False
)

show_all_checkbox.observe(show_all, 'value')


save_button = widgets.Button(
        description='Save isotherm',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
    )

save_button.on_click(save_button_clicked)

undo_button = widgets.Button(
        description='Undo',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
    )

undo_button.on_click(undo)

show_tracer_checkbox = widgets.Checkbox(
    value=False,
    description='Show tracer',
    disabled=False
)

show_tracer_checkbox.observe(show_tracer, 'value')

tracer_v = widgets.FloatLogSlider(
        value=1.0,
        min=v_limits[0],
        max=v_limits[1],
        step=abs(data.get('v')[0]-data.get('v')[1]),
        description=r'\( v \)',
        disabled=True,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
        layout = widgets.Layout(width = '90%')#, margin = '45px 0 0 0')
    )

tracer_v.observe(update_tracer, 'value')

label_input = widgets.Text(
    value='',
    placeholder="Label:",
    disabled = True,
)

add_label_button = widgets.Button(
        description='Add label',
        disabled=True,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip="Add label in tracer's position",
    )

add_label_button.on_click(add_label_button_clicked)

maxwell_construction_checkbox = widgets.Checkbox(
    value=False,
    description='Maxwell Construction',
    disabled=False
)

maxwell_construction_checkbox.observe(show_maxwell, 'value')

maxwell_p = widgets.FloatSlider(
        value=1.0,
        min=scale_y.min,
        max=scale_y.max,
        step=0.01,
        description=r'\( p_r \)',
        disabled=True,
        continuous_update=True,
        orientation='vertical',
        readout=True,
        readout_format='.2f',
        layout = widgets.Layout(height = '80%', margin = '10px 0 0 0')
    )

maxwell_p.observe(update_isopressure, 'value')

shade_areas_checkbox = widgets.Checkbox(
    value=False,
    description='shade areas',
    disabled=True
)

shade_areas_checkbox.observe(shade_areas, 'value')

integral_value_text = widgets.HTML(
    value="",
)

integral_value_left_text = widgets.HTML(
    value="",
) 

integral_value_right_text = widgets.HTML(
    value="",
) 

find_real_isotherm_button = widgets.Button(
        description='Find real isotherm',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
    )

find_real_isotherm_button.on_click(find_real_isotherm)

prepare_export_button = widgets.Button(
        description='Prepare to export',
        disabled=False,
        button_style='',
        tooltip='',
    )

prepare_export_button.on_click(prepare_export)

export_button = widgets.HTML(
    value = ""
)


#fig = update_figure(None)

widgets.VBox([
    widgets.HBox([
        widgets.VBox([widgets.HBox([a, widgets.HTMLMath(value=r'\( \text{J} \frac{m^3}{mol^2} \)')]),
                      widgets.HBox([b, widgets.HTMLMath(value=r'\( 10^{-3} \frac{m^3}{mol} \)')])]),
        widgets.VBox([widgets.HBox([t_min_input, widgets.HTMLMath(value=r'\( \quad T_c \)')]),
                      widgets.HBox([t_max_input, widgets.HTMLMath(value=r'\( \quad T_c \)')])])],
        layout = widgets.Layout(align_self = 'center')),

    widgets.HBox([widgets.Label(value="$p_c$:"), p_c_text, widgets.Label(value=r'\( \text{MPa} \qquad \)'),
                  widgets.Label(value="$v_c$:"), v_c_text, widgets.Label(value=r'\( \frac{cm^3}{mol} \qquad\)'),
                  widgets.Label(value="$T_c$:"), T_c_text, widgets.Label(value=r'\( \text{K} \qquad\)')
                 ], layout = widgets.Layout(align_self = 'center')),
    widgets.HBox([fig,
                  T_r,
                  widgets.VBox(
                      [show_all_checkbox, save_button, undo_button,],
                      layout = widgets.Layout(margin = '20% 0 0 0') )

                 ], layout = widgets.Layout(width = '100%')),
    widgets.HBox([
        widgets.VBox([show_tracer_checkbox,
                      tracer_v,
                     widgets.HBox([label_input, add_label_button], 
                                  layout = widgets.Layout(align_self = 'center')
                                 )],
                     layout = widgets.Layout(#align_self = 'center',
                         margin_left = '7px',
                         margin_right = '7px',
                         width='70%')),
        widgets.VBox([maxwell_construction_checkbox,
                      widgets.HBox([maxwell_p,
                                     widgets.VBox([shade_areas_checkbox,
                                                  widgets.HBox([widgets.Label(value="$Left:$"),
                                                                integral_value_left_text,]),
                                                  widgets.HBox([widgets.Label(value="$Right:$"),
                                                                integral_value_right_text,]),
                                                    widgets.HBox([widgets.Label(value="$Sum:$"),
                                                                integral_value_text,]),
                                                  find_real_isotherm_button,])
                        ],
                     layout = widgets.Layout(#align_self = 'center',
                                                                            margin_left = '7px')),
    ]), ]),
    widgets.HBox([prepare_export_button, export_button])
    ])
    #widgets.HBox([png_name, export_png_button,], layout = widgets.Layout(align_self = 'center',
                                                     #                   margin_bottom = '15px')),
    #widgets.HBox([png_created_text], )]), ])

ValueError: not enough values to unpack (expected 1, got 0)

In [37]:
top_block

In [38]:
get_t_range(t_min_input, t_max_input, num_input)

array([0.5       , 0.63636364, 0.77272727, 0.90909091, 1.04545455,
       1.18181818, 1.31818182, 1.45454545, 1.59090909, 1.72727273,
       1.86363636, 2.        ])